- Dataset
http://www2.informatik.uni-freiburg.de/~cziegler/BX/
- alt:
https://github.com/PetraWLeka/books_recommender_system/blob/master/BX-Books.csv

In [14]:
import pandas as pd

df = pd.read_csv('BX-Books.csv',encoding="latin1")
df.head()

C:\Users\PETRA LEKA\AppData\Local\Temp\ipykernel_29804\1295167678.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('BX-Books.csv',encoding="latin1")


,isbn,book_title,book_author,year_of_publication,publisher
0,195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,2005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,60973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271379 entries, 0 to 271378
Data columns (total 5 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   isbn                 271379 non-null  object
 1   book_title           271379 non-null  object
 2   book_author          271377 non-null  object
 3   year_of_publication  271379 non-null  object
 4   publisher            271377 non-null  object
dtypes: object(5)
memory usage: 10.4+ MB


In [16]:
#remove duplicate
df.duplicated(subset='book_title').sum()

29229

In [17]:
df = df.drop_duplicates(subset='book_title')

In [18]:

df.duplicated(subset='book_title').sum()

0

In [19]:
#Use only 15,000 rows instead of all 271,000 rows. It won't consume excessive memory.
sample_size = 15000
df = df.sample(n=sample_size, replace=False, random_state=42)

df = df.reset_index()
df = df.drop('index',axis=1)

In [20]:
df.head()

,isbn,book_title,book_author,year_of_publication,publisher
0,936399422,Norman Rockwell's Four Freedoms: Images That I...,Stuart Murray,1993,Berkshire House Publishers
1,055327726X,The Official Computer Freaks Joke Book,Larry Wilde,1988,Bantam Books
2,8445071629,"Costa Mas Lejana, La",Ursula K. Le Guin,1995,Minotauro
3,802448208,The Bull Fighter (Sugar Creek Gang),Paul Hutchins,1970,Moody Pr
4,451122720,Wild Harvest,Allison Mitchell,1983,New American Library


In [22]:
#convert text into its numeric representation before we can apply predictive modeling techniques onto it
#preprocess data
#combine name into one word
def clean_text(author):
    result = str(author).lower()
    return(result.replace(' ',''))
df['book_author'] = df['book_author'].apply(clean_text)

In [24]:
df['book_title'] = df['book_title'].str.lower()
df['publisher'] = df['publisher'].str.lower()

In [25]:
#combine these three columns to create a single variable
df2 = df.copy()
df2['data'] = df2[df2.columns[1:]].apply(
    lambda x: ' '.join(x.dropna().astype(str)),
    axis=1
)

print(df2['data'].head())

0    norman rockwell's four freedoms: images that i...
1    the official computer freaks joke book larrywi...
2    costa mas lejana, la ursulak.leguin 1995 minot...
3    the bull fighter (sugar creek gang) paulhutchi...
4    wild harvest allisonmitchell 1983 new american...
Name: data, dtype: object


In [26]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   isbn                 15000 non-null  object
 1   book_title           15000 non-null  object
 2   book_author          15000 non-null  object
 3   year_of_publication  15000 non-null  object
 4   publisher            15000 non-null  object
 5   data                 15000 non-null  object
dtypes: object(6)
memory usage: 703.2+ KB


In [27]:
# apply Scikit-Learn’s CountVectorizer() on the combined text data
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorized = vectorizer.fit_transform(df2['data'])

In [28]:
from sklearn.metrics.pairwise import cosine_similarity

similarities = cosine_similarity(vectorized)

In [29]:
print(similarities)

[[1.         0.         0.         ... 0.         0.09805807 0.        ]
 [0.         1.         0.         ... 0.09534626 0.1118034  0.        ]
 [0.         0.         1.         ... 0.         0.         0.        ]
 ...
 [0.         0.09534626 0.         ... 1.         0.10660036 0.        ]
 [0.09805807 0.1118034  0.         ... 0.10660036 1.         0.13363062]
 [0.         0.         0.         ... 0.         0.13363062 1.        ]]


In [30]:
df = pd.DataFrame(similarities, columns=df['book_title'], index=df['book_title']).reset_index()

df.head()

book_title,book_title,norman rockwell's four freedoms: images that inspire a nation,the official computer freaks joke book,"costa mas lejana, la",the bull fighter (sugar creek gang),wild harvest,"dorothy's world: childhood in sabine bottom, 1902-1910",just between us girls: secrets about men from the mayflower madam,"take up thy bed and walk: death, disability and cure in classic fiction for girls",lanark a life in books,...,here at the new yorker,"start with the soil: the organic gardener's guide to improving soil for higher yields, more beautiful flowers, and a healthy, easy-care garden",so long as you both shall live,should we burn babar?: essays on children's literature and the power of stories,"ann of the wild rose inn (wild rose inn, no 2)",princess school: who's the fairest,les cles du royaume,twinship: a novel by the author of ex utero,yiddish : a nation of words,weihnachten. groã?â?druck.
0,norman rockwell's four freedoms: images that i...,1.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.00000,0.0,...,0.000000,0.049814,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.098058,0.0
1,the official computer freaks joke book,0.0,1.0,0.0,0.1,0.0,0.08165,0.079057,0.06742,0.2,...,0.105409,0.113592,0.087706,0.074536,0.145095,0.1,0.0,0.095346,0.111803,0.0
2,"costa mas lejana, la",0.0,0.0,1.0,0.0,0.0,0.00000,0.000000,0.00000,0.0,...,0.000000,0.000000,0.000000,0.083333,0.000000,0.0,0.0,0.000000,0.000000,0.0
3,the bull fighter (sugar creek gang),0.0,0.1,0.0,1.0,0.0,0.00000,0.079057,0.06742,0.0,...,0.105409,0.170389,0.000000,0.074536,0.072548,0.1,0.0,0.095346,0.000000,0.0
4,wild harvest,0.0,0.0,0.0,0.0,1.0,0.00000,0.000000,0.00000,0.0,...,0.125988,0.000000,0.209657,0.089087,0.173422,0.0,0.0,0.000000,0.000000,0.0


In [34]:
df['book_title']

0        norman rockwell's four freedoms: images that i...
1                   the official computer freaks joke book
2                                     costa mas lejana, la
3                      the bull fighter (sugar creek gang)
4                                             wild harvest
                               ...                        
14995                   princess school: who's the fairest
14996                                  les cles du royaume
14997          twinship: a novel by the author of ex utero
14998                          yiddish : a nation of words
14999                           weihnachten. groã?â?druck.
Name: book_title, Length: 15000, dtype: object

In [35]:
input_book = "princess school: who's the fairest"
recommendations = pd.DataFrame(df.nlargest(11,input_book)['book_title'])
recommendations = recommendations[recommendations['book_title']!=input_book]
print(recommendations)

                                              book_title
13149  the shortstop who knew too much (tales from th...
5815               a room in the attic (princess , no 2)
12254                          the tortoise and the hare
12394                             the girl in the window
3108                                             the fog
3352                                     the candy witch
7235                                        the castaway
12077                                the mummy scrapbook
13643                                    the ghostmobile
5188                     the girl on the outside (point)
